In [8]:
from collections import Counter, defaultdict
import pickle as pkl
import matplotlib.pyplot as plt
import itertools
import numpy as np
import pandas as pd

%matplotlib notebook

In [9]:
with open("./error_analysis_json/generative_cf-v0_tf-v2.pkl", "rb") as fr:
    dict_outputs_generative_tfv2 = pkl.load(fr)
with open("./error_analysis_json/generative_paper.pkl", "rb") as fr:
    dict_outputs_generative = pkl.load(fr)
with open("./error_analysis_json/discriminative_paper.pkl", "rb") as fr:
    dict_outputs_discriminative = pkl.load(fr)

In [10]:
def pretty_print(examples, results_path, seen_keys=None, background_keys=None):
    with open(results_path, "w") as fw:
        for example_idx, example in enumerate(examples):
            if not seen_keys and not background_keys:
                str_print = "Example %i/%i: " % (example_idx + 1, len(examples))
            else:
                str_print = "Example %i/%i (%s, full background: %r)" % (example_idx + 1, 
                                                                        len(examples),
                                                                        seen_keys[example_idx],
                                                                        background_keys[example_idx])
            str_print += "\n\nEntities (%s): %s" % (example["category"], ', '.join(example["entities"]))
            str_print += "\n\nContext:\n%s" % example["instance"]
            str_print += "\n\nScores:"
    

            for score_name in example["scores"]:
                str_print += "\n%s: %.3f" % (score_name, example["scores"][score_name])

            results = zip(example["ranks"], 
                          example["choices"], 
                          example["preds"], 
                          example["targets"])
            results = sorted(results, key=lambda x: x[0])
            
            str_print += "\n\nGold standards (rank: choice [output/target]:"
   
            for result in results:
                if result[3]:
                    str_print += "\n%i: %s [%.3f/%i]" % result

            str_print += "\n\nTop %i results (rank: choice [output/target]:" % len(example["choices"])
            for result in results:
                str_print += "\n%i: %s [%.3f/%i]" % result

            str_print += "\n\nTop %i Generations (sorted by probability):" % len(example["generations"])
            
            if example["generations"][0] is not None:
                for idx, (generation, proba) in enumerate(example["generations"]):
                     str_print += "\n%i: %s [%.3f]" % (idx, generation, proba)
                 
            str_print += "\n\n--------------------------------------------------" \
                         "-------------------------------------------------- \n\n"
            
            print(str_print)
            fw.write(str_print)

In [21]:
data_loader_key = "test"
seen_key = "seen"
category_key = "person"

In [22]:
# Muestras donde el MAP de generative sea mejor que el de generative_tfv2

c = 0
for i in range(len(dict_outputs_generative[data_loader_key][seen_key][category_key])):
    d1 = dict_outputs_discriminative[data_loader_key][seen_key][category_key][i]
    d2 = dict_outputs_generative_tfv2[data_loader_key][seen_key][category_key][i]
    names = ["Discriminative", "Generative-2"]
    if d1["scores"]["average_precision"] > d2["scores"]["average_precision"]:
        print("Example %i" % c)
        c += 1
        str_print = ""
        str_print += "\n\nEntities (%s): %s" % (d1["category"], ', '.join(d1["entities"]))
        str_print += "\n\nContext:\n%s" % d1["instance"]
        str_print += "\n\nScores:"
        for score_name in d1["scores"]:
            str_print += "\n%s: %.3f | %.3f" % (score_name, d1["scores"][score_name], d2["scores"][score_name])
        results_d1 = zip(d1["ranks"], 
                         d1["choices"], 
                         d1["preds"], 
                         d1["targets"])
        results_d2 = zip(d2["ranks"], 
                 d2["choices"], 
                 d2["preds"], 
                 d2["targets"])
        results_d1 = sorted(results_d1, key=lambda x: x[0])
        results_d2 = sorted(results_d2, key=lambda x: x[0])
        str_print += "\n\nGold standards (D1)"
        for result_d1 in results_d1:
            if result_d1[3]:
                str_print += "\n%i: %s [%.3f/%i]" % result_d1
        str_print += "\n"*2 + "-" * 50 + "\n" + "Gold standards (D2)"
        for result_d2 in results_d2:
            if result_d2[3]:
                str_print += "\n%i: %s [%.3f/%i]" % result_d2
                
        table_data = []

        for result_d1, result_d2 in zip(results_d1, results_d2):
            table_data.append(("%s [%.3f/%i]" % result_d1[1:], "%s [%.3f/%i]" % result_d2[1:]))
            
        str_print += "\n\nGenerations:"
        for idx, (generation, proba) in enumerate(d2["generations"]):
            str_print += "\n%i: %s [%.3f]" % (idx, generation, proba)
            
        str_print += "\n\nTop %i results (rank: choice [output/target]:" % len(d1["choices"])
        
        print(str_print)
        
        df = pd.DataFrame(table_data, columns=names)
        display(df)
            
        print("\n"*2 + "-"*50+"\n"*2)

Example 0


Entities (person): Adrian Garcia, Paul Capdeville

Context:
Adrian Garcia is the former Sheriff of Harris County, Texas. Garcia spent 23 years with the Houston Police Department, before becoming a city councilman. Garcia served six years as a Houston City Councilman, eventually becoming Mayor Pro-Tempore under former Mayor Bill White. In 2008 he became Sheriff of Harris County. In 2015 Garcia stepped down from post as Sheriff to run for Mayor of the City of Houston.Garcia won the general election for Harris County Commissioners Court Precinct 2 on November 6, 2018, after advancing from the primary on March 6, 2018. Paul Gerard Capdeville Castro is a Chilean former tennis player. He was born in Santiago. U.S. Wins Doubles To Take A 2-1 Lead: He could not resist a little humor, saying,'' You know, I threw them a bone on that point.'' Capdeville and Garcia had played only one other Tour-level match together this year, and they lost it, too. Adrian Garcia, Paul Capdeville

Scor

,Discriminative,Generative-2
0,players [0.978/1],players [0.849/1]
1,men of hispanic descent [0.949/1],businessmen [0.847/0]
2,people involved with the law [0.212/0],american professionals [0.833/0]
3,dead [0.000/0],executives [0.832/0]
4,memorializers of the american offensive [0.000/0],contractors [0.831/0]
5,american professionals [0.000/0],men of hispanic descent [0.827/1]
6,men involved in the angela correa case [0.000/0],cooks [0.824/0]
7,philantropists [0.000/0],american officials [0.820/0]
8,involved in a potentially criminal scheme [0.0...,people involved with the law [0.820/0]
9,businessmen [0.000/0],kenyans [0.816/0]




--------------------------------------------------


Example 1


Entities (person): Jennifer Ehle, Tom Stoppard

Context:
Jennifer Anne Ehle is a British-American actress. She won the BAFTA TV Award for Best Actress for her role as Elizabeth Bennet in the 1995 BBC miniseries Pride and Prejudice. For her work on Broadway, she won the 2000 Tony Award for Best Actress in a Play for The Real Thing, and the 2007 Tony Award for Best Featured Actress in a Play for The Coast of Utopia. She is the daughter of English actress Rosemary Harris and American author John Ehle. Sir Tom Stoppard  is a Czech-born British playwright and screenwriter. He has written prolifically for TV, radio, film and stage, finding prominence with plays such as Arcadia, The Coast of Utopia, Every Good Boy Deserves Favour, Professional Foul, The Real Thing, Travesties, The Invention of Love, and Rosencrantz and Guildenstern Are Dead. He co-wrote the screenplays for Brazil, The Russia House, and Shakespeare in Love, and

,Discriminative,Generative-2
0,entertainers [0.998/1],entertainers [0.888/1]
1,entertainment industry professionals [0.807/0],entertainment industry professionals [0.875/0]
2,people in show business [0.764/1],film directors [0.874/0]
3,hollywood celebrities [0.557/0],hollywood celebrities [0.869/0]
4,dead [0.043/0],suspects [0.869/0]
5,film directors [0.001/0],people in show business [0.867/1]
6,co-authors [0.000/0],family members [0.864/0]
7,conspirators [0.000/0],co-authors [0.862/0]
8,suspects [0.000/0],conspirators [0.861/0]
9,aspects of the german film world [0.000/0],fashion designers [0.860/0]




--------------------------------------------------


Example 2


Entities (person): Dan Gerstein, Joseph I. Lieberman, Sherry Brown

Context:
Joseph Isadore Lieberman is an American politician, lobbyist and attorney who served as a United States Senator from Connecticut from 1989 to 2013. A former member of the Democratic Party, he was its nominee for Vice President of the United States in the 2000 election. During his final term in office he was officially listed as an independent Democrat and caucused with and chaired committees for the Democratic Party. Feeling 'Very Good' About Giving Himself Another Chance: Mr. Lieberman also hired his former aide , Dan Gerstein , as communications director for the campaign. Mr. Gerstein and Ms. Brown both worked as senior advisers on Mr. Lieberman 's campaigns when he ran for vice president in 2000 and for president in 2004.'' Dan Gerstein, Joseph I. Lieberman, Sherry Brown

Scores:
average_precision: 0.375 | 0.303
precision_at_10: 0.200 | 0.10

...



--------------------------------------------------


Example 62


Entities (person): Robert Grenier, Valerie Plame Wilson

Context:
Robert L. Grenier is a longtime CIA officer who served as the CIA's top counter-terrorism official and was fired from that position by CIA director Porter Goss. Valerie Elise Plame Wilson , is an American writer, spy novelist, and former officer who worked at the United States Central Intelligence Agency . As the subject of the 2003 Plame affair, also known as the CIA leak scandal, Plame's identity as a CIA officer was leaked to and subsequently published by Robert Novak of the Washington Post. Ex-Officials Testify They Informed Libby of the Identity of a C.I.A. Operative: Mr. Grenier said he had not remembered that part of the conversation with Mr. Libby and only recalled it more than a year later when he went before a grand jury again. With a generous dollop of scorn, Mr. Jeffress asked why Mr. Grenier had first told the grand jury that he had no recol

,Discriminative,Generative-2
0,cia officers [0.997/1],cia officers [0.859/1]
1,spy workers [0.973/1],hollywood figures [0.840/0]
2,intelligence analysts [0.850/0],intelligence analysts [0.840/0]
3,represenatives [0.001/0],spy workers [0.839/1]
4,skiers [0.000/0],muslim men [0.834/0]
5,former comptroller candidates [0.000/0],broadcasters [0.828/0]
6,jock ewing [0.000/0],american democrats [0.828/0]
7,muslim men [0.000/0],skiers [0.824/0]
8,american democrats [0.000/0],religious people [0.821/0]
9,english-speaking entertainers [0.000/0],fashion designers [0.820/0]




--------------------------------------------------


Example 63


Entities (person): Hillary Rodham Clinton, John Spencer

Context:
Hillary Diane Rodham Clinton is an American politician, diplomat, lawyer, writer and public speaker. She was First Lady of the United States from 1993 to 2001, a United States senator from New York from 2001 to 2009, and the 67th United States secretary of state from 2009 until 2013. Clinton became the first woman to be nominated for president of the United States by a major political party when she won the Democratic Party nomination in 2016. She was the first woman to win the popular vote in an American presidential election. Clinton Faces Questions About Future During Lively Debate: You know, in the middle of the Civil War, Abraham Lincoln did not hesitate to change generals,'' Mrs. Clinton said, drawing applause from the audience at the University of Rochester auditorium.'' You 're not president yet, Mrs. Clinton ,'' her opponent , John Spencer , the

,Discriminative,Generative-2
0,politicians [0.998/1],politicians [0.857/1]
1,opponents [0.966/1],leaders [0.839/0]
2,secretaries [0.746/0],american officials [0.833/0]
3,american officials [0.269/0],opponents [0.830/1]
4,leaders [0.115/0],american figures [0.827/0]
5,american figures [0.006/0],italians [0.824/0]
6,counselors [0.000/0],married couple [0.821/0]
7,married couple [0.000/0],supreme court justices [0.812/0]
8,persons of interest [0.000/0],foundation members [0.810/0]
9,opposing parties in the enron legal proceeding...,counselors [0.810/0]




--------------------------------------------------


